In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
import MetaTrader5 as mt5
# from vectorbt.portfolio import Portfolio as vbt_Portfolio

# Connection

In [ ]:
# RICO_REAL   = { 'login':16331371,   'password':'#582640Zz', 'server':'Rico-PRD'                 }
# CLEAR_REAL  = { 'login':1018156111, 'password':'#582640Zz', 'server':'ClearInvestimentos-CLEAR' }

# PATH_7 = r'C:\Program Files\MetaTrader 5 _7\terminal64.exe'

# assert mt5.initialize(PATH_7, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$'] # ['WDO$']
TFRAMES = ['M5','M15','H1','H3','D1']

# Load

In [ ]:
# def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
#     Pipe = []
#     for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
#         timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
#         Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
#         Raw.insert(0, 'a',          Raw.index + 1)
#         Raw.insert(1, 'z',          Raw.index - Raw.index.max())
#         Raw.insert(2, 'symbol',     symbol)
#         Raw.insert(3, 'tf',         TF)
#         Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
#         Pipe.append(Raw)
#     pass
#     return pd.concat(Pipe, ignore_index=1)

# Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
# Raw.head()

In [ ]:
# Raw.to_csv('Storage/Raw.csv', index=False)
Raw = pd.read_csv('Storage/Raw.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$'], TFRAMES=['M15'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']   = Src['Close'] - Src['Open']
    Src['HL']       = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    def _int(x):
        try:    return int(x)
        except: return x
        
    Calc = pd.DataFrame(Src)
    Calc['Day Open']        = Calc.groupby(['Symbol','TF','Date'], sort=0)['Open']  .transform('first')
    Calc['Day High']        = Calc.groupby(['Symbol','TF','Date'], sort=0)['High']  .transform('max')
    Calc['Day Low']         = Calc.groupby(['Symbol','TF','Date'], sort=0)['Low']   .transform('min')
    Calc['Day Close']       = Calc.groupby(['Symbol','TF','Date'], sort=0)['Close'] .transform('last')
    Calc['Day Chg']         = Calc['Day Close'] - Calc['Day Open']
    Calc['Day Chg Abs']     = Calc['Day Chg'].abs()

    Calc['Intra High Rank'] = Calc.groupby(['Symbol','TF','Date'], sort=0)['High'].rank(method='dense', ascending=0)
    Calc['Intra Low Rank']  = Calc.groupby(['Symbol','TF','Date'], sort=0)['Low'].rank(method='dense', ascending=1)

    WINS = [0, 100, 200, 500, 800, 1000, 1200, 1500, 1800, 2000, 2200, 2500, 2800, 3000, 3200, 3500, 3800, 4000, 4200, 4500, 4800, 5000, inf]
    for (a,b) in [x for x in pd.Series(WINS).rolling(2) if (len(x) == 2)]:
        Calc[f'Day Chg Abs {_int(a)}-{_int(b)}'] = ((a <= Calc['Day Chg Abs']) & (Calc['Day Chg Abs'] < b)) .astype(int)

    return Calc

Calc = CALCULATIONS(Src)
Calc.head(2)

In [82]:
def STATS(Calc, bins):
    pipe = []
    for x in bins: 
        Df1 = Calc[Calc[x]==1] #.reset_index(drop=1)

        for y in ['High','Low']:
            Df2 = Df1[Df1[f'Intra {y} Rank']==1] 

            pipe.append({ 'Bins':x, 'HL':y,
                'Count':    Df2['Time'].count(),
                'Time Avg': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).mean(), 1),
                'Time Max': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).max(), 1),
                'Time Q3':  np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).quantile(.75), 1),
                'Time Med': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).median(), 1),
                'Time Q1':  np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).quantile(.25), 1),
                'Time Min': np.round(Df2['Time'].apply(lambda x: x.hour + x.minute/60).min(), 1),
            })
        pass
    pass
    return pd.DataFrame(pipe)


Sts = STATS(Calc, bins=Calc.loc[:, f'Day Chg Abs 0-100':f'Day Chg Abs 5000-inf'].columns)
Sts

,Bins,HL,Count,Time Avg,Time Max,Time Q3,Time Med,Time Q1,Time Min
0,Day Chg Abs 0-100,High,63,12.4,18.2,14.0,11.8,10.5,9.0
1,Day Chg Abs 0-100,Low,64,11.7,18.2,12.5,11.0,9.7,9.0
2,Day Chg Abs 100-200,High,71,12.4,17.8,14.1,11.8,10.4,9.0
3,Day Chg Abs 100-200,Low,70,11.8,18.2,13.4,11.0,10.2,9.0
4,Day Chg Abs 200-500,High,182,12.3,18.2,14.2,11.8,10.2,9.0
5,Day Chg Abs 200-500,Low,180,11.9,18.2,13.3,11.2,10.2,9.0
6,Day Chg Abs 500-800,High,161,12.2,18.2,14.8,11.2,10.0,9.0
7,Day Chg Abs 500-800,Low,159,12.4,18.2,14.4,12.0,10.5,9.0
8,Day Chg Abs 800-1000,High,134,13.0,18.2,15.9,12.2,10.3,9.0
9,Day Chg Abs 800-1000,Low,135,12.2,18.2,14.4,11.5,9.9,9.0
